In [ ]:
import synapseclient

import great_expectations as gx

context = gx.get_context(project_root_dir='../src/agoradatatools/great_expectations')

# Create Expectation Suite for Proteomics SRM Data

## Get Example Data File

In [ ]:
syn = synapseclient.Synapse()
syn.login()


In [ ]:
proteomics_srm_data_file = syn.get("syn52579910").path

## Create Validator Object on Data File

In [ ]:
validator = context.sources.pandas_default.read_json(
    proteomics_srm_data_file
)
validator.expectation_suite_name = "proteomics_srm"


## Add Expectations to Validator Object For Each Column

In [ ]:
# uniqid
validator.expect_column_values_to_be_of_type("uniqid", "str")
validator.expect_column_values_to_not_be_null("uniqid")
validator.expect_column_values_to_match_regex("uniqid", "^\\w{2,}|[A-NR-Z][0-9]([A-Z][A-Z0-9]{2}[0-9]){1,2}$")

In [ ]:
# hgnc_symbol
validator.expect_column_values_to_be_of_type("hgnc_symbol", "str")
validator.expect_column_value_lengths_to_be_between("hgnc_symbol", 1, 15)
validator.expect_column_values_to_match_regex("hgnc_symbol", "^[A-Za-z0-9_.-]+$")
validator.expect_column_values_to_not_be_null("hgnc_symbol")

In [ ]:
# uniprotid
validator.expect_column_values_to_be_of_type("uniprotid", "str")
validator.expect_column_value_lengths_to_be_between("uniprotid", 1, 15)
validator.expect_column_values_to_match_regex("uniprotid", "^[a-zA-Z0-9.-]*$")
validator.expect_column_values_to_not_be_null("uniprotid")

In [ ]:
# ensembl_gene_id
validator.expect_column_values_to_be_of_type("ensembl_gene_id", "str")
validator.expect_column_values_to_not_be_null("ensembl_gene_id")
validator.expect_column_values_to_match_regex("ensembl_gene_id", "^ENSG\d{11}$")

In [ ]:
# tissue
validator.expect_column_values_to_be_of_type("tissue", "str")
validator.expect_column_values_to_not_be_null("tissue")
validator.expect_column_values_to_be_in_set("tissue", ["DLPFC"])

In [ ]:
# log2_fc
validator.expect_column_values_to_be_of_type("log2_fc", "float")

In [ ]:
# ci_upr
validator.expect_column_values_to_be_of_type("ci_upr", "float")

In [ ]:
# ci_lwr
validator.expect_column_values_to_be_of_type("ci_lwr", "float")

In [ ]:
# pval
validator.expect_column_values_to_be_of_type("pval", "float")
validator.expect_column_values_to_be_between("pval", 0, 1)

In [ ]:
# cor_pval
validator.expect_column_values_to_be_of_type("cor_pval", "float")
validator.expect_column_values_to_be_between("cor_pval", 0, 1)

In [ ]:
# multi-field
validator.expect_compound_columns_to_be_unique(["hgnc_symbol", "uniprotid", "tissue"])
validator.expect_compound_columns_to_be_unique(["ensembl_gene_id", "uniprotid", "tissue"])

## Save Expectation Suite

In [ ]:
validator.save_expectation_suite(discard_failed_expectations=False)


## Create Checkpoint and View Results

In [ ]:
checkpoint = context.add_or_update_checkpoint(
    name="agora-test-checkpoint",
    validator=validator,
)
checkpoint_result = checkpoint.run()
context.view_validation_result(checkpoint_result)


## Build Data Docs - Click on Expectation Suite to View All Expectations

In [ ]:
context.build_data_docs()
context.open_data_docs()
